In [1]:
'''
    remote_exec
    IBM/Dell Qiskit Runtime Emulator Demo
    July 26, 2021
    
    
    The following notebook walks through a (simple) example usage of the 
    Qiskit Runtime Emulator in a remote execution environment: i.e. one in
    a Kubernetes cluster or deployed anywhere that is not the client machine.
'''


from qiskit_emulator import EmulatorProvider
from qiskit import QuantumCircuit
import logging
import requests
import time
import os

RUNTIME_PROGRAM = """
# This code is part of qiskit-runtime.
#
# (C) Copyright IBM 2021.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.
from qiskit.compiler import transpile, schedule


def main(
    backend,
    user_messenger,
    circuits,
    **kwargs,
):
    circuits = transpile(
        circuits,
    )
    
    user_messenger.publish({'results': 'intermittently'}, final=False)

    if not isinstance(circuits, list):
        circuits = [circuits]

    # Compute raw results
    result = backend.run(circuits, **kwargs).result()

    user_messenger.publish(result.to_dict(), final=True)
"""

RUNTIME_PROGRAM_METADATA = {
    "max_execution_time": 600,
    "description": "Qiskit test program"
}

PROGRAM_PREFIX = 'qiskit-test'
REMOTE_RUNTIME = os.getenv("SERVER_URL") 

logging.basicConfig(level=logging.DEBUG)

In [2]:
'''
    The main difference between the local and remote runtime interfaces
    is here: `provider.remote(URL_OF_REMOTE)`
    
    This function connects the client to the remote environment and, if 
    necessary, logs them in through SSO if enabled. 
'''

provider = EmulatorProvider()
provider.remote(REMOTE_RUNTIME)

INFO:qiskit.compiler.assembler:Total Assembly Time - 0.08130 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.05889 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.05507 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.05341 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.05317 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.05293 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.05364 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.05198 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06127 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06819 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.05817 (ms)
DEBUG:qiskit_emulator.remote_runtime_service:GET http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/status
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80
DEBUG:urllib3.connectionpool:http://qre-dev.oro-san


Your QRE ID is: 32e10b11b642.
Save this ID and run 'export QRE_ID=<your id>' to access your programs in future sessions.


In [3]:
'''
    Uploading a program works exactly the same as locally
'''

text_program_id = provider.runtime.upload_program(RUNTIME_PROGRAM, metadata=RUNTIME_PROGRAM_METADATA)
print(f"PROGRAM ID: {text_program_id}")

DEBUG:qiskit_emulator.remote_runtime_service:POST http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program: {'data': "\n# This code is part of qiskit-runtime.\n#\n# (C) Copyright IBM 2021.\n#\n# This code is licensed under the Apache License, Version 2.0. You may\n# obtain a copy of this license in the LICENSE.txt file in the root directory\n# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.\n#\n# Any modifications or derivative works of this code must retain this\n# copyright notice, and modified files need to carry a notice indicating\n# that they have been altered from the originals.\nfrom qiskit.compiler import transpile, schedule\n\n\ndef main(\n    backend,\n    user_messenger,\n    circuits,\n    **kwargs,\n):\n    circuits = transpile(\n        circuits,\n    )\n    \n    user_messenger.publish({'results': 'intermittently'}, final=False)\n\n    if not isinstance(circuits, list):\n        circuits = [circuits]\n\n    # Compute raw results\n    result = 

PROGRAM ID: a0f36f6045e2


In [4]:
programs = provider.runtime.pprint_programs(refresh=True)

DEBUG:qiskit_emulator.remote_runtime_service:GET http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /program HTTP/1.1" 200 151
DEBUG:qiskit_emulator.remote_runtime_service:GET http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program RESPONSE: (200, 'OK', '[{"program_id": "a0f36f6045e2", "name": "a0f36f6045e2", "program_metadata": "{\\"max_execution_time\\": 600, \\"description\\": \\"Qiskit test program\\"}"}]')


a0f36f6045e2:
  Name: a0f36f6045e2
  Description: Qiskit test program
  Version: 0
  Creation date: 
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none


In [5]:
provider.runtime.update_program(text_program_id, description="IBM/Dell Updated Qiskit Runtime Program")

DEBUG:qiskit_emulator.remote_runtime_service:POST http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program/a0f36f6045e2/update: {'program_id': 'a0f36f6045e2', 'data': None, 'name': None, 'data_type': None, 'program_metadata': '{"description": "IBM/Dell Updated Qiskit Runtime Program"}'}
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "POST /program/a0f36f6045e2/update HTTP/1.1" 200 0
DEBUG:qiskit_emulator.remote_runtime_service:POST http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program/a0f36f6045e2/update RESPONSE: (200, 'OK', '')
DEBUG:qiskit_emulator.remote_runtime_service:Successfully updated program a0f36f6045e2


True

In [6]:
programs = provider.runtime.pprint_programs(refresh=True)

DEBUG:qiskit_emulator.remote_runtime_service:GET http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /program HTTP/1.1" 200 171
DEBUG:qiskit_emulator.remote_runtime_service:GET http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program RESPONSE: (200, 'OK', '[{"program_id": "a0f36f6045e2", "name": "a0f36f6045e2", "program_metadata": "{\\"max_execution_time\\": 600, \\"description\\": \\"IBM/Dell Updated Qiskit Runtime Program\\"}"}]')


a0f36f6045e2:
  Name: a0f36f6045e2
  Description: IBM/Dell Updated Qiskit Runtime Program
  Version: 0
  Creation date: 
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none


In [7]:
'''
    This can also be done locally - but if the provided string to upload_program
    is a file or directory path, the upload function will grab the contents 
    and send those to the remote orchestrator.
'''


file_program_id = provider.runtime.upload_program("qka.py", description="File Upload to Orchestrator")

DEBUG:qiskit_emulator.remote_runtime_service:POST http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program: {'data': None, 'name': None, 'data_type': 'STRING', 'program_metadata': '{"description": "File Upload to Orchestrator"}'}
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "POST /program HTTP/1.1" 200 12
DEBUG:qiskit_emulator.remote_runtime_service:POST http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program RESPONSE: (200, 'OK', '86d015902548')
DEBUG:qiskit_emulator.remote_runtime_service:Received program_id: 86d015902548


In [8]:
dir_program_id = provider.runtime.upload_program("./qkad", description="Directory Upload to Orchestrator")

DEBUG:qiskit_emulator.remote_runtime_service:Have directory: ./qkad
DEBUG:qiskit_emulator.remote_runtime_service:made: /home/geoff/workspace/qiskit-runtime-emulator/examples/demo_notebooks/qkad.zip
DEBUG:qiskit_emulator.remote_runtime_service:POST http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program: {'data': None, 'name': None, 'data_type': 'DIR', 'program_metadata': '{"description": "Directory Upload to Orchestrator"}'}
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "POST /program HTTP/1.1" 200 12
DEBUG:qiskit_emulator.remote_runtime_service:POST http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program RESPONSE: (200, 'OK', '4de208d58e5a')
DEBUG:qiskit_emulator.remote_runtime_service:Received program_id: 4de208d58e5a


In [9]:
provider.runtime.pprint_programs(refresh=True)

DEBUG:qiskit_emulator.remote_runtime_service:GET http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /program HTTP/1.1" 200 436
DEBUG:qiskit_emulator.remote_runtime_service:GET http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program RESPONSE: (200, 'OK', '[{"program_id": "a0f36f6045e2", "name": "a0f36f6045e2", "program_metadata": "{\\"max_execution_time\\": 600, \\"description\\": \\"IBM/Dell Updated Qiskit Runtime Program\\"}"}, {"program_id": "86d015902548", "name": "86d015902548", "program_metadata": "{\\"description\\": \\"File Upload to Orchestrator\\"}"}, {"program_id": "4de208d58e5a", "name": "4de208d58e5a", "program_metadata": "{\\"description\\": \\"Directory Upload to Orchestrator\\"}"}]')


a0f36f6045e2:
  Name: a0f36f6045e2
  Description: IBM/Dell Updated Qiskit Runtime Program
  Version: 0
  Creation date: 
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none
86d015902548:
  Name: 86d015902548
  Description: File Upload to Orchestrator
  Version: 0
  Creation date: 
  Max execution time: 0
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none
4de208d58e5a:
  Name: 4de208d58e5a
  Description: Directory Upload to Orchestrator
  Version: 0
  Creation date: 
  Max execution time: 0
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none


In [10]:
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

program_inputs = {
    'circuits': qc,
}

In [11]:
job = provider.runtime.run(text_program_id, options=None, inputs=program_inputs)

DEBUG:qiskit_emulator.remote_runtime_service:POST http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program/a0f36f6045e2/job: {"circuits": {"__type__": "dill", "__value__": "gASVSQYAAAAAAACMHXFpc2tpdC5jaXJjdWl0LnF1YW50dW1jaXJjdWl0lIwOUXVhbnR1bUNpcmN1aXSUk5QpgZR9lCiMCl9iYXNlX25hbWWUjAdjaXJjdWl0lIwEbmFtZZSMCWNpcmN1aXQtMpSMBV9kYXRhlF2UKIwncWlza2l0LmNpcmN1aXQubGlicmFyeS5zdGFuZGFyZF9nYXRlcy5olIwFSEdhdGWUk5QpgZR9lCiMBl9sYWJlbJROjAtfZGVmaW5pdGlvbpROaAeMAWiUjApudW1fcXViaXRzlEsBjApudW1fY2xiaXRzlEsAjAdfcGFyYW1zlF2UjAljb25kaXRpb26UTowJX2R1cmF0aW9ulE6MBV91bml0lIwCZHSUdWJdlIwecWlza2l0LmNpcmN1aXQucXVhbnR1bXJlZ2lzdGVylIwFUXViaXSUk5QpgZROfZQojAVfcmVwcpSMIVF1Yml0KFF1YW50dW1SZWdpc3RlcigyLCAncScpLCAwKZSMBl9pbmRleJRLAIwFX2hhc2iUigioebecehKesIwJX3JlZ2lzdGVylGgcjA9RdWFudHVtUmVnaXN0ZXKUk5QpgZROfZQojAVfbmFtZZSMAXGUjAVfc2l6ZZRLAowFX2JpdHOUXZQoaB9oHimBlE59lChoIYwhUXViaXQoUXVhbnR1bVJlZ2lzdGVyKDIsICdxJyksIDEplGgjSwFoJIoIAeEHhkTb7QpoJWgodYaUYmVoJIoItmvIVWh8JaFoIYwXUXVhbnR1bVJlZ2lzdGVyKDIsICdxJymUdYaUYnWGlGJhXZ

In [12]:
results = job.result(timeout=60)
print(results)

{'backend_name': 'aer_simulator', 'backend_version': '0.8.2', 'qobj_id': '8ecb892e-b07e-4fad-b485-ba0c2eac471e', 'job_id': 'c6217605-c571-4f32-a578-c49d6389ec2e', 'success': True, 'results': [{'shots': 1024, 'success': True, 'data': {'counts': {'0x3': 513, '0x0': 511}}, 'meas_level': 2, 'header': {'clbit_labels': [['c', 0], ['c', 1]], 'creg_sizes': [['c', 2]], 'global_phase': 0.0, 'memory_slots': 2, 'metadata': None, 'n_qubits': 2, 'name': 'circuit-2', 'qreg_sizes': [['q', 2]], 'qubit_labels': [['q', 0], ['q', 1]]}, 'status': 'DONE', 'seed_simulator': 3703703757, 'metadata': {'parallel_state_update': 4, 'measure_sampling': True, 'method': 'stabilizer', 'parallel_shots': 1, 'device': 'CPU', 'fusion': {'enabled': False}}, 'time_taken': 0.0042886700000000005}], 'date': '2021-08-04T13:01:18.701041', 'status': 'COMPLETED', 'header': {'backend_name': 'aer_simulator', 'backend_version': '0.8.2'}, 'metadata': {'time_taken': 0.004665226000000001, 'parallel_experiments': 1, 'omp_enabled': True, 

In [13]:
def callback_function(msg):
    print(f'******************\n\n\nFrom Callback Function: {msg}\n\n\n******************')
    

In [14]:
program_inputs['backend_name'] = 'emulator'
job = provider.runtime.run(dir_program_id, inputs=program_inputs, options=None, callback=callback_function)

DEBUG:qiskit_emulator.remote_runtime_service:POST http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/program/4de208d58e5a/job: {"circuits": {"__type__": "dill", "__value__": "gASVSQYAAAAAAACMHXFpc2tpdC5jaXJjdWl0LnF1YW50dW1jaXJjdWl0lIwOUXVhbnR1bUNpcmN1aXSUk5QpgZR9lCiMCl9iYXNlX25hbWWUjAdjaXJjdWl0lIwEbmFtZZSMCWNpcmN1aXQtMpSMBV9kYXRhlF2UKIwncWlza2l0LmNpcmN1aXQubGlicmFyeS5zdGFuZGFyZF9nYXRlcy5olIwFSEdhdGWUk5QpgZR9lCiMBl9sYWJlbJROjAtfZGVmaW5pdGlvbpROaAeMAWiUjApudW1fcXViaXRzlEsBjApudW1fY2xiaXRzlEsAjAdfcGFyYW1zlF2UjAljb25kaXRpb26UTowJX2R1cmF0aW9ulE6MBV91bml0lIwCZHSUdWJdlIwecWlza2l0LmNpcmN1aXQucXVhbnR1bXJlZ2lzdGVylIwFUXViaXSUk5QpgZROfZQojAVfcmVwcpSMIVF1Yml0KFF1YW50dW1SZWdpc3RlcigyLCAncScpLCAwKZSMBl9pbmRleJRLAIwFX2hhc2iUigioebecehKesIwJX3JlZ2lzdGVylGgcjA9RdWFudHVtUmVnaXN0ZXKUk5QpgZROfZQojAVfbmFtZZSMAXGUjAVfc2l6ZZRLAowFX2JpdHOUXZQoaB9oHimBlE59lChoIYwhUXViaXQoUXVhbnR1bVJlZ2lzdGVyKDIsICdxJyksIDEplGgjSwFoJIoIAeEHhkTb7QpoJWgodYaUYmVoJIoItmvIVWh8JaFoIYwXUXVhbnR1bVJlZ2lzdGVyKDIsICdxJymUdYaUYnWGlGJhXZ

In [15]:
results = job.result(timeout=600)
print(results)

DEBUG:qiskit_emulator.emulator_runtime_job:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/job/f4e149efe0be/results
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /job/f4e149efe0be/results HTTP/1.1" 200 16
DEBUG:qiskit_emulator.emulator_runtime_job:got: {'messages': []}
DEBUG:qiskit_emulator.emulator_runtime_job:result: got []
DEBUG:qiskit_emulator.emulator_runtime_job:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/job/f4e149efe0be/status
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /job/f4e149efe0be/status HTTP/1.1" 200 7
DEBUG:qiskit_emulator.emulator_runtime_job:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/job/f4e149efe0be/results
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /job/f4e149efe0be/results HTTP/1.1" 200 16
DEBUG:qiskit_emulator.emulator_runtime_job:got: {'messages': []}
DEBUG:qiskit_emulator.emulator_runtime_job:resu

******************


From Callback Function: {'cost': 11.855372765048024, 'kernel_parameters': {'__type__': 'array', '__value__': [0.24730511661879886]}}


******************


DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /job/f4e149efe0be/status HTTP/1.1" 200 7
DEBUG:qiskit_emulator.emulator_runtime_job:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/job/f4e149efe0be/results
DEBUG:qiskit_emulator.emulator_runtime_job:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/job/f4e149efe0be/results/2021-08-04 13:02:04
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /job/f4e149efe0be/results/2021-08-04%2013:02:04 HTTP/1.1" 200 16
DEBUG:qiskit_emulator.emulator_runtime_job:got: {'messages': []}
DEBUG:qiskit_emulator.emulator_runtime_job:result: got []
DEBUG:qiskit_emulator.emulator_runtime_job:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/job/f4e149efe0be/status
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /job/f4e149efe0be/status HTTP/1.1" 200 7
DEBUG:qiskit_emulator.emulator_runtime_job:http://qre-dev.oro-sandbox-sm

DEBUG:qiskit_emulator.emulator_runtime_job:result: got [{'data': '{"final": true, "message": {"aligned_kernel_parameters": {"__type__": "array", "__value__": [0.24730511661879886]}, "aligned_kernel_matrix": {"__type__": "array", "__value__": [[1.0, 0.0023193359375, 0.004150390625, 0.052490234375, 0.059814453125, 0.0003662109375, 0.000244140625, 0.003662109375, 0.005859375, 0.052978515625, 0.004638671875, 0.0225830078125, 0.0623779296875, 0.0115966796875, 0.0029296875, 0.0146484375, 0.6761474609375, 0.002685546875, 0.2115478515625, 0.046630859375], [0.0023193359375, 1.0, 0.0, 0.0003662109375, 0.00048828125, 0.0001220703125, 0.071533203125, 0.0076904296875, 0.003662109375, 0.056884765625, 0.0225830078125, 0.0009765625, 0.0003662109375, 0.3590087890625, 0.03271484375, 0.005126953125, 0.0048828125, 0.0224609375, 0.032958984375, 0.0732421875], [0.004150390625, 0.0, 1.0, 0.0673828125, 0.0616455078125, 0.0472412109375, 0.0001220703125, 0.0040283203125, 0.00244140625, 0.0001220703125, 0.0, 0.0

{'aligned_kernel_parameters': {'__type__': 'array', '__value__': [0.24730511661879886]}, 'aligned_kernel_matrix': {'__type__': 'array', '__value__': [[1.0, 0.0023193359375, 0.004150390625, 0.052490234375, 0.059814453125, 0.0003662109375, 0.000244140625, 0.003662109375, 0.005859375, 0.052978515625, 0.004638671875, 0.0225830078125, 0.0623779296875, 0.0115966796875, 0.0029296875, 0.0146484375, 0.6761474609375, 0.002685546875, 0.2115478515625, 0.046630859375], [0.0023193359375, 1.0, 0.0, 0.0003662109375, 0.00048828125, 0.0001220703125, 0.071533203125, 0.0076904296875, 0.003662109375, 0.056884765625, 0.0225830078125, 0.0009765625, 0.0003662109375, 0.3590087890625, 0.03271484375, 0.005126953125, 0.0048828125, 0.0224609375, 0.032958984375, 0.0732421875], [0.004150390625, 0.0, 1.0, 0.0673828125, 0.0616455078125, 0.0472412109375, 0.0001220703125, 0.0040283203125, 0.00244140625, 0.0001220703125, 0.0, 0.0001220703125, 0.000244140625, 0.0, 0.0, 0.0, 0.0079345703125, 0.0, 0.0010986328125, 0.0], [0.

DEBUG:qiskit_emulator.emulator_runtime_job:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/job/f4e149efe0be/results
DEBUG:qiskit_emulator.emulator_runtime_job:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com/job/f4e149efe0be/results/2021-08-04 13:02:19
DEBUG:urllib3.connectionpool:http://qre-dev.oro-sandbox-small1.k8s.cec.lab.emc.com:80 "GET /job/f4e149efe0be/results/2021-08-04%2013:02:19 HTTP/1.1" 200 16
DEBUG:qiskit_emulator.emulator_runtime_job:got: {'messages': []}
DEBUG:qiskit_emulator.emulator_runtime_job:result: got []
